# Function Calling with Azure OpenAI
- https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling


## [Parallel function calling](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling#parallel-function-calling-with-multiple-functions)
- gpt-35-turbo (1106)
- gpt-35-turbo (0125)
- gpt-4 (1106-Preview)
- gpt-4 (0125-Preview)
- gpt-4 (vision-preview)
- gpt-4 (2024-04-09)
- gpt-4o (2024-05-13)
- gpt-4o-mini (2024-07-18)

Support for parallel function was first added in API version 2023-12-01-preview

## [Single function calling with tools](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling#single-toolfunction-calling-example)

All the models that support parallel function calling
- gpt-4 (0613)
- gpt-4-32k (0613)
- gpt-35-turbo-16k (0613)
- gpt-35-turbo (0613)


In [1]:
import os
import json
from openai import AzureOpenAI
from datetime import datetime
from zoneinfo import ZoneInfo

from dotenv import load_dotenv

# Get Configuration Settings
load_dotenv()

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = os.getenv("AOAI_ENDPOINT"), 
    api_key=os.getenv("AOAI_KEY"),  
    api_version="2024-06-01"
)

# Define the deployment you want to use for your chat completions API calls
deployment_name = "gpt-4o-mini"


In [2]:
# Reusable Functions
# Simplified weather data
WEATHER_DATA = {
    "tokyo": {"temperature": "10", "unit": "celsius"},
    "san francisco": {"temperature": "72", "unit": "fahrenheit"},
    "paris": {"temperature": "22", "unit": "celsius"}
}

# Simplified timezone data
TIMEZONE_DATA = {
    "tokyo": "Asia/Tokyo",
    "san francisco": "America/Los_Angeles",
    "paris": "Europe/Paris"
}

def get_current_weather(location, unit=None):
    """Get the current weather for a given location"""
    # print(f"get_current_weather called with location: {location}, unit: {unit}")  
    location_lower = location.lower()
    
    for key in WEATHER_DATA:
        if key in location_lower:
            # print(f"Weather data found for {key}")  
            weather = WEATHER_DATA[key]
            return json.dumps({
                "location": location,
                "temperature": weather["temperature"],
                "unit": unit if unit else weather["unit"]
            })
    
    # print(f"No weather data found for {location_lower}")  
    return json.dumps({"location": location, "temperature": "unknown"})

def get_current_time(location):
    """Get the current time for a given location"""
    # print(f"get_current_time called with location: {location}")  
    location_lower = location.lower()
    
    for key, timezone in TIMEZONE_DATA.items():
        if key in location_lower:
            # print(f"Timezone found for {key}")  
            current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
            return json.dumps({
                "location": location,
                "current_time": current_time
            })
    
    # print(f"No timezone data found for {location_lower}")  
    return json.dumps({"location": location, "current_time": "unknown"})

# Define the functions for the model
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g. San Francisco",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g. San Francisco",
                    },
                },
                "required": ["location"],
            },
        }
    }
]

def run_conversation(content:str):
    # Initial user message
    messages = [{"role": "user", "content": content}]

    # First API call: Ask the model to use the functions
    response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=TOOLS,
        tool_choice="auto",
    )

    # Process the model's response
    response_message = response.choices[0].message
    messages.append(response_message)

    print("Model's response:")  
    print(response_message)  

    # Handle function calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function call: {function_name}")  
            print(f"Function arguments: {function_args}")  
            if function_name == "get_current_weather":
                function_response = get_current_weather(
                    location=function_args.get("location"),
                    unit=function_args.get("unit")
                )
            elif function_name == "get_current_time":
                function_response = get_current_time(
                    location=function_args.get("location")
                )
            else:
                function_response = json.dumps({"error": "Unknown function"})
            
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
    else:
        print("No tool calls were made by the model.")  

    # Second API call: Get the final response from the model
    final_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
    )

    return final_response.choices[0].message.content

In [3]:
# Run the conversation and print the result
print(run_conversation("What's the current time in San Francisco"))

Model's response:
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ydopjPOwPFsBxN6QRfkc5jTs', function=Function(arguments='{"location":"San Francisco"}', name='get_current_time'), type='function')], refusal=None)
Function call: get_current_time
Function arguments: {'location': 'San Francisco'}
The current time in San Francisco is 6:27 AM.


In [4]:
print(run_conversation("What's the current time in San Francisco, Tokyo, and Paris?"))


Model's response:
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hqtdtT0qCsM2gVo3PZdoHRdv', function=Function(arguments='{"location": "San Francisco"}', name='get_current_time'), type='function'), ChatCompletionMessageToolCall(id='call_LmKpXWblJ2ihXuViaaoYXTd5', function=Function(arguments='{"location": "Tokyo"}', name='get_current_time'), type='function'), ChatCompletionMessageToolCall(id='call_6onutUpENwlSyF2H8v7GazZ9', function=Function(arguments='{"location": "Paris"}', name='get_current_time'), type='function')], refusal=None)
Function call: get_current_time
Function arguments: {'location': 'San Francisco'}
Function call: get_current_time
Function arguments: {'location': 'Tokyo'}
Function call: get_current_time
Function arguments: {'location': 'Paris'}
As of now, the current times are as follows:

- **San Francisco**: 05:33 AM
- **Tokyo**: 10:33 PM
- **Paris**: 02:33 PM


In [5]:
print(run_conversation("What's the weather and current time in San Francisco, Tokyo, and Paris?"))

Model's response:
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yZEBZyuA9QpyXD7XSahke8Ry', function=Function(arguments='{"location": "San Francisco"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_wwPr10xPapq05WEmv6vka038', function=Function(arguments='{"location": "Tokyo"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_30CP6rurBXKeOAGNbUForc98', function=Function(arguments='{"location": "Paris"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_1b6dNy6WKCdiGg9nvnZgWA1o', function=Function(arguments='{"location": "San Francisco"}', name='get_current_time'), type='function'), ChatCompletionMessageToolCall(id='call_4H9jrLJNx4bi5dYvclWozq2m', function=Function(arguments='{"location": "Tokyo"}', name='get_current_time'), type='function'), ChatCompletionMessageToolCall(id='call_Z1QFh6yGmKo854RlF

In [7]:
# Run the conversation and print the result
print(run_conversation("What's the weather and current time in San Francisco, Tokyo, and Paris?"))

Model's response:
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yDOa7Fb6Wq5qcUzyPOfg44O0', function=Function(arguments='{"location": "San Francisco"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_oHqgWoc8MWwGAEkLOVmp4SOc', function=Function(arguments='{"location": "Tokyo"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_bqXv92hFY7fEqdAw1aNMl2Ma', function=Function(arguments='{"location": "Paris"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_wVl2jA3KycqG2DIJnJ28O8BX', function=Function(arguments='{"location": "San Francisco"}', name='get_current_time'), type='function'), ChatCompletionMessageToolCall(id='call_pdXYKHfxyEgI4opqzEjkPUEG', function=Function(arguments='{"location": "Tokyo"}', name='get_current_time'), type='function'), ChatCompletionMessageToolCall(id='call_VVtp6RwDu91SjLRX5

In [28]:
print(run_conversation("What's the current time in San Francisco, Tokyo, and Paris?"))

Model's response:
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XIhdT6ZGkAYYT0mY067MyK7y', function=Function(arguments='{"location": "San Francisco"}', name='get_current_time'), type='function'), ChatCompletionMessageToolCall(id='call_ynEqmqu33abdvO9uKXMIXhe6', function=Function(arguments='{"location": "Tokyo"}', name='get_current_time'), type='function'), ChatCompletionMessageToolCall(id='call_zvJ2MQVQr5K7jmtxgea0lxpj', function=Function(arguments='{"location": "Paris"}', name='get_current_time'), type='function')])
Function arguments: {'location': 'San Francisco'}
Function arguments: {'location': 'Tokyo'}
Function arguments: {'location': 'Paris'}
The current time is:
- San Francisco: 10:51 AM
- Tokyo: 02:51 AM
- Paris: 07:51 PM
